In [1]:
import torch
import math
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
import torch.nn.functional as F
import time
import os
import torch.backends.cudnn as cudnn

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'                # GPU Number 
start_time = time.time()
batch_size = 128
learning_rate = 0.001
root_dir = 'D:/Confidential Reports/Deep learning class/assignment'
default_directory = 'D:/Confidential Reports/Deep learning class/assignment/save_models'

In [5]:
# Data Augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),               # Random Position Crop
    transforms.RandomHorizontalFlip(),                  # right and left flip
    transforms.ToTensor(),                              # change [0,255] Int value to [0,1] Float value
    transforms.Normalize(mean=(0.4914, 0.4824, 0.4467), # RGB Normalize MEAN
                         std=(0.2471, 0.2436, 0.2616))  # RGB Normalize Standard Deviation
])

transform_test = transforms.Compose([
    transforms.ToTensor(),                              # change [0,255] Int value to [0,1] Float value
    transforms.Normalize(mean=(0.4914, 0.4824, 0.4467), # RGB Normalize MEAN
                         std=(0.2471, 0.2436, 0.2616))  # RGB Normalize Standard Deviation
])


In [6]:
# automatically download
train_dataset = datasets.CIFAR10(root=root_dir,
                                 train=True,
                                 transform=transform_train,
                                 download=True)

test_dataset = datasets.CIFAR10(root=root_dir,
                                train=False,
                                transform=transform_test)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,            # at Training Procedure, Data Shuffle = True
                                           num_workers=4)           # CPU loader number

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False,            # at Test Procedure, Data Shuffle = False
                                          num_workers=4)            # CPU loader number


Extracting D:/Confidential Reports/Deep learning class/assignment\cifar-10-python.tar.gz to D:/Confidential Reports/Deep learning class/assignment


In [7]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = out + self.shortcut(x)
        out = F.relu(out)
        return out

In [8]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [9]:
model = ResNet(BasicBlock, [2, 2, 2, 2])
optimizer = optim.SGD(model.parameters(), learning_rate,
                                momentum=0.9,
                                weight_decay=1e-4,
                                nesterov=True)
criterion = nn.CrossEntropyLoss()

if torch.cuda.device_count() > 0:
    print("USE", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model).cuda()
    cudnn.benchmark = True
else:
    print("USE ONLY CPU!")

USE ONLY CPU!


D:\Anaconda\lib\site-packages\torch\cuda\__init__.py:82: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  C:\cb\pytorch_1000000000000\work\c10\cuda\CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [10]:
def train(epoch):
    model.train()
    train_loss = 0 
    total = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        if torch.cuda.is_available():
            data, target = Variable(data.cuda()), Variable(target.cuda())
        else:
            data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(output.data, 1)

        total += target.size(0)
        correct += predicted.eq(target.data).cpu().sum()
        if batch_idx % 10 == 0:
            print('Epoch: {} | Batch_idx: {} |  Loss: ({:.4f}) | Acc: ({:.2f}%) ({}/{})'
                  .format(epoch, batch_idx, train_loss / (batch_idx + 1), 100. * correct / total, correct, total))



In [11]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        if torch.cuda.is_available():
            data, target = Variable(data.cuda()), Variable(target.cuda())
        else:
            data, target = Variable(data), Variable(target)

        outputs = model(data)
        loss = criterion(outputs, target)

        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += predicted.eq(target.data).cpu().sum()
    print('# TEST : Loss: ({:.4f}) | Acc: ({:.2f}%) ({}/{})'
          .format(test_loss / (batch_idx + 1), 100. * correct / total, correct, total))

In [12]:
def save_checkpoint(directory, state, filename='latest.tar.gz'):

    if not os.path.exists(directory):
        os.makedirs(directory)

    model_filename = os.path.join(directory, filename)
    torch.save(state, model_filename)
    print("=> saving checkpoint")

In [13]:
def load_checkpoint(directory, filename='latest.tar.gz'):

    model_filename = os.path.join(directory, filename)
    if os.path.exists(model_filename):
        print("=> loading checkpoint")
        state = torch.load(model_filename)
        return state
    else:
        return None


In [14]:
start_epoch = 0

checkpoint = load_checkpoint(default_directory)
if not checkpoint:
    pass
else:
    start_epoch = checkpoint['epoch'] + 1
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

In [15]:
for epoch in range(start_epoch, 165):

    if epoch < 80:
        lr = learning_rate
    elif epoch < 120:
        lr = learning_rate * 0.1
    else:
        lr = learning_rate * 0.01
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    train(epoch)
    save_checkpoint(default_directory, {
        'epoch': epoch,
        'model': model,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
    })
    test()  

now = time.gmtime(time.time() - start_time)
print('{} hours {} mins {} secs for training'.format(now.tm_hour, now.tm_min, now.tm_sec))

Epoch: 0 | Batch_idx: 0 |  Loss: (2.6346) | Acc: (9.38%) (12/128)
Epoch: 0 | Batch_idx: 10 |  Loss: (2.3526) | Acc: (14.77%) (208/1408)
Epoch: 0 | Batch_idx: 20 |  Loss: (2.2697) | Acc: (17.15%) (461/2688)
Epoch: 0 | Batch_idx: 30 |  Loss: (2.2028) | Acc: (19.08%) (757/3968)
Epoch: 0 | Batch_idx: 40 |  Loss: (2.1595) | Acc: (20.18%) (1059/5248)
Epoch: 0 | Batch_idx: 50 |  Loss: (2.1190) | Acc: (22.32%) (1457/6528)
Epoch: 0 | Batch_idx: 60 |  Loss: (2.0816) | Acc: (23.98%) (1872/7808)
Epoch: 0 | Batch_idx: 70 |  Loss: (2.0565) | Acc: (24.78%) (2252/9088)
Epoch: 0 | Batch_idx: 80 |  Loss: (2.0338) | Acc: (25.57%) (2651/10368)
Epoch: 0 | Batch_idx: 90 |  Loss: (2.0127) | Acc: (26.30%) (3063/11648)
Epoch: 0 | Batch_idx: 100 |  Loss: (1.9948) | Acc: (27.04%) (3496/12928)
Epoch: 0 | Batch_idx: 110 |  Loss: (1.9780) | Acc: (27.69%) (3934/14208)
Epoch: 0 | Batch_idx: 120 |  Loss: (1.9638) | Acc: (28.21%) (4369/15488)
Epoch: 0 | Batch_idx: 130 |  Loss: (1.9460) | Acc: (28.88%) (4842/16768)
Epoc

Epoch: 2 | Batch_idx: 310 |  Loss: (1.2285) | Acc: (55.59%) (22129/39808)
Epoch: 2 | Batch_idx: 320 |  Loss: (1.2269) | Acc: (55.64%) (22861/41088)
Epoch: 2 | Batch_idx: 330 |  Loss: (1.2258) | Acc: (55.73%) (23613/42368)
Epoch: 2 | Batch_idx: 340 |  Loss: (1.2247) | Acc: (55.76%) (24339/43648)
Epoch: 2 | Batch_idx: 350 |  Loss: (1.2225) | Acc: (55.85%) (25093/44928)
Epoch: 2 | Batch_idx: 360 |  Loss: (1.2218) | Acc: (55.90%) (25832/46208)
Epoch: 2 | Batch_idx: 370 |  Loss: (1.2198) | Acc: (55.98%) (26584/47488)
Epoch: 2 | Batch_idx: 380 |  Loss: (1.2177) | Acc: (56.08%) (27350/48768)
Epoch: 2 | Batch_idx: 390 |  Loss: (1.2159) | Acc: (56.13%) (28066/50000)
=> saving checkpoint
# TEST : Loss: (1.1989) | Acc: (56.56%) (5656/10000)
Epoch: 3 | Batch_idx: 0 |  Loss: (1.1087) | Acc: (62.50%) (80/128)
Epoch: 3 | Batch_idx: 10 |  Loss: (1.0944) | Acc: (61.72%) (869/1408)
Epoch: 3 | Batch_idx: 20 |  Loss: (1.1246) | Acc: (60.19%) (1618/2688)
Epoch: 3 | Batch_idx: 30 |  Loss: (1.1383) | Acc: (5

Epoch: 5 | Batch_idx: 210 |  Loss: (0.9467) | Acc: (65.90%) (17799/27008)
Epoch: 5 | Batch_idx: 220 |  Loss: (0.9444) | Acc: (66.01%) (18674/28288)
Epoch: 5 | Batch_idx: 230 |  Loss: (0.9447) | Acc: (66.02%) (19522/29568)
Epoch: 5 | Batch_idx: 240 |  Loss: (0.9439) | Acc: (66.08%) (20385/30848)
Epoch: 5 | Batch_idx: 250 |  Loss: (0.9428) | Acc: (66.09%) (21233/32128)
Epoch: 5 | Batch_idx: 260 |  Loss: (0.9433) | Acc: (66.09%) (22080/33408)
Epoch: 5 | Batch_idx: 270 |  Loss: (0.9437) | Acc: (66.04%) (22908/34688)
Epoch: 5 | Batch_idx: 280 |  Loss: (0.9444) | Acc: (66.05%) (23756/35968)
Epoch: 5 | Batch_idx: 290 |  Loss: (0.9436) | Acc: (66.11%) (24626/37248)
Epoch: 5 | Batch_idx: 300 |  Loss: (0.9427) | Acc: (66.19%) (25503/38528)
Epoch: 5 | Batch_idx: 310 |  Loss: (0.9418) | Acc: (66.27%) (26381/39808)
Epoch: 5 | Batch_idx: 320 |  Loss: (0.9416) | Acc: (66.30%) (27242/41088)
Epoch: 5 | Batch_idx: 330 |  Loss: (0.9425) | Acc: (66.30%) (28090/42368)
Epoch: 5 | Batch_idx: 340 |  Loss: (0.

Epoch: 8 | Batch_idx: 110 |  Loss: (0.8153) | Acc: (71.01%) (10089/14208)
Epoch: 8 | Batch_idx: 120 |  Loss: (0.8127) | Acc: (71.14%) (11018/15488)
Epoch: 8 | Batch_idx: 130 |  Loss: (0.8129) | Acc: (71.12%) (11926/16768)
Epoch: 8 | Batch_idx: 140 |  Loss: (0.8120) | Acc: (71.09%) (12830/18048)
Epoch: 8 | Batch_idx: 150 |  Loss: (0.8114) | Acc: (71.25%) (13772/19328)
Epoch: 8 | Batch_idx: 160 |  Loss: (0.8096) | Acc: (71.30%) (14694/20608)
Epoch: 8 | Batch_idx: 170 |  Loss: (0.8068) | Acc: (71.39%) (15625/21888)
Epoch: 8 | Batch_idx: 180 |  Loss: (0.8053) | Acc: (71.43%) (16549/23168)
Epoch: 8 | Batch_idx: 190 |  Loss: (0.8055) | Acc: (71.47%) (17474/24448)
Epoch: 8 | Batch_idx: 200 |  Loss: (0.8046) | Acc: (71.48%) (18391/25728)
Epoch: 8 | Batch_idx: 210 |  Loss: (0.8049) | Acc: (71.47%) (19302/27008)
Epoch: 8 | Batch_idx: 220 |  Loss: (0.8039) | Acc: (71.51%) (20229/28288)
Epoch: 8 | Batch_idx: 230 |  Loss: (0.8020) | Acc: (71.60%) (21172/29568)
Epoch: 8 | Batch_idx: 240 |  Loss: (0.

# TEST : Loss: (0.8007) | Acc: (71.98%) (7198/10000)
Epoch: 11 | Batch_idx: 0 |  Loss: (0.5799) | Acc: (79.69%) (102/128)
Epoch: 11 | Batch_idx: 10 |  Loss: (0.7196) | Acc: (74.72%) (1052/1408)
Epoch: 11 | Batch_idx: 20 |  Loss: (0.7080) | Acc: (75.04%) (2017/2688)
Epoch: 11 | Batch_idx: 30 |  Loss: (0.7061) | Acc: (75.43%) (2993/3968)
Epoch: 11 | Batch_idx: 40 |  Loss: (0.7113) | Acc: (74.92%) (3932/5248)
Epoch: 11 | Batch_idx: 50 |  Loss: (0.7017) | Acc: (75.43%) (4924/6528)
Epoch: 11 | Batch_idx: 60 |  Loss: (0.7052) | Acc: (75.15%) (5868/7808)
Epoch: 11 | Batch_idx: 70 |  Loss: (0.7072) | Acc: (75.18%) (6832/9088)
Epoch: 11 | Batch_idx: 80 |  Loss: (0.7069) | Acc: (75.14%) (7791/10368)
Epoch: 11 | Batch_idx: 90 |  Loss: (0.7067) | Acc: (75.14%) (8752/11648)
Epoch: 11 | Batch_idx: 100 |  Loss: (0.7068) | Acc: (75.06%) (9704/12928)
Epoch: 11 | Batch_idx: 110 |  Loss: (0.7029) | Acc: (75.18%) (10682/14208)
Epoch: 11 | Batch_idx: 120 |  Loss: (0.7019) | Acc: (75.13%) (11636/15488)
Epoc

...

Epoch: 153 | Batch_idx: 270 |  Loss: (0.0532) | Acc: (98.53%) (34179/34688)
Epoch: 153 | Batch_idx: 280 |  Loss: (0.0528) | Acc: (98.55%) (35446/35968)
Epoch: 153 | Batch_idx: 290 |  Loss: (0.0530) | Acc: (98.54%) (36703/37248)
Epoch: 153 | Batch_idx: 300 |  Loss: (0.0531) | Acc: (98.53%) (37961/38528)
Epoch: 153 | Batch_idx: 310 |  Loss: (0.0534) | Acc: (98.52%) (39218/39808)
Epoch: 153 | Batch_idx: 320 |  Loss: (0.0534) | Acc: (98.52%) (40479/41088)
Epoch: 153 | Batch_idx: 330 |  Loss: (0.0534) | Acc: (98.51%) (41737/42368)
Epoch: 153 | Batch_idx: 340 |  Loss: (0.0532) | Acc: (98.52%) (43003/43648)
Epoch: 153 | Batch_idx: 350 |  Loss: (0.0532) | Acc: (98.52%) (44264/44928)
Epoch: 153 | Batch_idx: 360 |  Loss: (0.0531) | Acc: (98.53%) (45527/46208)
Epoch: 153 | Batch_idx: 370 |  Loss: (0.0532) | Acc: (98.52%) (46785/47488)
Epoch: 153 | Batch_idx: 380 |  Loss: (0.0529) | Acc: (98.53%) (48052/48768)
Epoch: 153 | Batch_idx: 390 |  Loss: (0.0534) | Acc: (98.51%) (49254/50000)
=> saving ch

Epoch: 156 | Batch_idx: 140 |  Loss: (0.0528) | Acc: (98.49%) (17776/18048)
Epoch: 156 | Batch_idx: 150 |  Loss: (0.0534) | Acc: (98.47%) (19033/19328)
Epoch: 156 | Batch_idx: 160 |  Loss: (0.0533) | Acc: (98.46%) (20290/20608)
Epoch: 156 | Batch_idx: 170 |  Loss: (0.0531) | Acc: (98.48%) (21555/21888)
Epoch: 156 | Batch_idx: 180 |  Loss: (0.0528) | Acc: (98.51%) (22822/23168)
Epoch: 156 | Batch_idx: 190 |  Loss: (0.0530) | Acc: (98.51%) (24083/24448)
Epoch: 156 | Batch_idx: 200 |  Loss: (0.0531) | Acc: (98.51%) (25344/25728)
Epoch: 156 | Batch_idx: 210 |  Loss: (0.0530) | Acc: (98.53%) (26611/27008)
Epoch: 156 | Batch_idx: 220 |  Loss: (0.0527) | Acc: (98.54%) (27876/28288)
Epoch: 156 | Batch_idx: 230 |  Loss: (0.0532) | Acc: (98.52%) (29129/29568)
Epoch: 156 | Batch_idx: 240 |  Loss: (0.0533) | Acc: (98.51%) (30389/30848)
Epoch: 156 | Batch_idx: 250 |  Loss: (0.0538) | Acc: (98.49%) (31642/32128)
Epoch: 156 | Batch_idx: 260 |  Loss: (0.0537) | Acc: (98.49%) (32904/33408)
Epoch: 156 |

Epoch: 159 | Batch_idx: 0 |  Loss: (0.0733) | Acc: (97.66%) (125/128)
Epoch: 159 | Batch_idx: 10 |  Loss: (0.0521) | Acc: (98.72%) (1390/1408)
Epoch: 159 | Batch_idx: 20 |  Loss: (0.0527) | Acc: (98.74%) (2654/2688)
Epoch: 159 | Batch_idx: 30 |  Loss: (0.0496) | Acc: (98.94%) (3926/3968)
Epoch: 159 | Batch_idx: 40 |  Loss: (0.0506) | Acc: (98.74%) (5182/5248)
Epoch: 159 | Batch_idx: 50 |  Loss: (0.0511) | Acc: (98.70%) (6443/6528)
Epoch: 159 | Batch_idx: 60 |  Loss: (0.0514) | Acc: (98.66%) (7703/7808)
Epoch: 159 | Batch_idx: 70 |  Loss: (0.0521) | Acc: (98.58%) (8959/9088)
Epoch: 159 | Batch_idx: 80 |  Loss: (0.0520) | Acc: (98.62%) (10225/10368)
Epoch: 159 | Batch_idx: 90 |  Loss: (0.0526) | Acc: (98.59%) (11484/11648)
Epoch: 159 | Batch_idx: 100 |  Loss: (0.0532) | Acc: (98.55%) (12740/12928)
Epoch: 159 | Batch_idx: 110 |  Loss: (0.0528) | Acc: (98.54%) (14001/14208)
Epoch: 159 | Batch_idx: 120 |  Loss: (0.0533) | Acc: (98.52%) (15259/15488)
Epoch: 159 | Batch_idx: 130 |  Loss: (0.0

Epoch: 161 | Batch_idx: 270 |  Loss: (0.0528) | Acc: (98.54%) (34183/34688)
Epoch: 161 | Batch_idx: 280 |  Loss: (0.0525) | Acc: (98.55%) (35448/35968)
Epoch: 161 | Batch_idx: 290 |  Loss: (0.0524) | Acc: (98.58%) (36719/37248)
Epoch: 161 | Batch_idx: 300 |  Loss: (0.0524) | Acc: (98.58%) (37980/38528)
Epoch: 161 | Batch_idx: 310 |  Loss: (0.0522) | Acc: (98.58%) (39242/39808)
Epoch: 161 | Batch_idx: 320 |  Loss: (0.0523) | Acc: (98.57%) (40500/41088)
Epoch: 161 | Batch_idx: 330 |  Loss: (0.0523) | Acc: (98.57%) (41763/42368)
Epoch: 161 | Batch_idx: 340 |  Loss: (0.0528) | Acc: (98.56%) (43018/43648)
Epoch: 161 | Batch_idx: 350 |  Loss: (0.0527) | Acc: (98.57%) (44285/44928)
Epoch: 161 | Batch_idx: 360 |  Loss: (0.0529) | Acc: (98.57%) (45547/46208)
Epoch: 161 | Batch_idx: 370 |  Loss: (0.0529) | Acc: (98.57%) (46811/47488)
Epoch: 161 | Batch_idx: 380 |  Loss: (0.0531) | Acc: (98.56%) (48065/48768)
Epoch: 161 | Batch_idx: 390 |  Loss: (0.0531) | Acc: (98.56%) (49280/50000)
=> saving ch

Epoch: 164 | Batch_idx: 130 |  Loss: (0.0552) | Acc: (98.34%) (16489/16768)
Epoch: 164 | Batch_idx: 140 |  Loss: (0.0549) | Acc: (98.38%) (17756/18048)
Epoch: 164 | Batch_idx: 150 |  Loss: (0.0545) | Acc: (98.40%) (19019/19328)
Epoch: 164 | Batch_idx: 160 |  Loss: (0.0543) | Acc: (98.39%) (20277/20608)
Epoch: 164 | Batch_idx: 170 |  Loss: (0.0541) | Acc: (98.41%) (21540/21888)
Epoch: 164 | Batch_idx: 180 |  Loss: (0.0542) | Acc: (98.42%) (22801/23168)
Epoch: 164 | Batch_idx: 190 |  Loss: (0.0540) | Acc: (98.43%) (24065/24448)
Epoch: 164 | Batch_idx: 200 |  Loss: (0.0543) | Acc: (98.43%) (25323/25728)
Epoch: 164 | Batch_idx: 210 |  Loss: (0.0544) | Acc: (98.42%) (26582/27008)
Epoch: 164 | Batch_idx: 220 |  Loss: (0.0543) | Acc: (98.43%) (27843/28288)
Epoch: 164 | Batch_idx: 230 |  Loss: (0.0542) | Acc: (98.45%) (29109/29568)
Epoch: 164 | Batch_idx: 240 |  Loss: (0.0541) | Acc: (98.46%) (30374/30848)
Epoch: 164 | Batch_idx: 250 |  Loss: (0.0538) | Acc: (98.48%) (31639/32128)
Epoch: 164 |